# Project: Wrangling and Analyze Data

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#DG">Data Gathering</a></li>
<li><a href="#AD">Assessing Data</a></li>
<li><a href="#CD">Cleaning Data</a></li>
<li><a href="#SD">Storing Data</a></li>
<li><a href="#AVD">Analyzing and Visualizing Data</a></li>
</ul>

<a id='intro'></a>
## Introduction

This project is a Data wrangling project which will be based on WeRateDogs Twitter data.
This project's goal is to gather data from a various sources and in various forms, assess its quality and tidiness, and then do the necessary cleaning to prepare it for insightful analysis and visualizations.

In [ ]:
# importing every necessary library

import json
import requests
from functools import reduce
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

<a id='DG'></a>
## Data Gathering
All three pieces of data needed for this project were obtained and loaded into the notebook in the cells below.

In [ ]:
# Reading the twitter-archive-enhanced file
# saving it as a dataframe with the name dog_rate

dog_rate = pd.read_csv('twitter-archive-enhanced.csv')

In [ ]:
# displaying few top rows from the dog_rate dataframe

dog_rate.head()

In [ ]:
# displaying some information about the dog_rate dataframe

dog_rate.info()

In [ ]:
dog_rate.name.value_counts()

In [ ]:
# Viewing the count of the unique values in rating_numerator column

dog_rate.rating_numerator.value_counts()

In [ ]:
# Viewing the count of the unique values in rating_denominator column

dog_rate.rating_denominator.value_counts()

In [ ]:
# Checking for duplicates in the dog_rate dataframe

dog_rate.duplicated().sum()

2. Use the Requests library to download the tweet image prediction (image_predictions.tsv)

In [ ]:
# saving the web address of image_predictions file to a variable
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

# Use the Requests library to download the tweet image prediction file using the url address
r = requests.get(url, allow_redirects=True)

In [ ]:
# Checking for the status of the request sent

r.status_code

In [ ]:
# Checking for the content type of the file downloaded

r.headers.get('content-type')

In [ ]:
# Writing the file into my local computer

open('image-predictions.tsv', 'wb').write(r.content)

In [ ]:
# Reading the image-predictions file
# saving it as a dataframe with the name image_predictions

image_predictions = pd.read_csv('image-predictions.tsv', sep = '\t')

In [ ]:
# displaying few top rows from the image_predictions dataframe

image_predictions.head()

In [ ]:
# displaying some information about the image_predictions dataframe

image_predictions.info()

In [ ]:
# Checking for duplicates in the image_predictions dataframe

image_predictions.duplicated().sum()

3. Use the Tweepy library to query additional data via the Twitter API (tweet_json.txt)

In [ ]:
# Reading a text file line by line
# using the text to form a list of dictionaries

with open('tweet-json.txt') as file:
    tweets = []
    for line in file:
        tweets.append(json.loads(line))

In [ ]:
# Create a dataframe from tweet_list of dictionaries
# saving it as a dataframe with the name tweets_df

tweets_df = pd.DataFrame(tweets, columns = ['id', 'retweet_count', 'favorite_count'])

In [ ]:
# displaying few top rows from the tweets_df dataframe

tweets_df.head()

In [ ]:
# displaying some information about the tweets_df dataframe

tweets_df.info()

In [ ]:
# Checking for duplicates in the tweets_df dataframe

tweets_df.duplicated().sum()

<a id='AD'></a>
## Assessing Data
In this section, at least eight (8) quality issues and two (2) tidiness issue were found and documented. Evaluating the data through both programmatic and visual assessment.

### Quality issues
#### dog_rate table
1. incorrect datatype in the timestamp column.
2. No need for the replies and retweet rows.
3. No need for the replies and retweet columns.
4. In rating_denominator, some values are less than 10.

#### image_predictions table
5. The names in the P columns have some lowercase and some uppercase initials.
6. The conf should be written completely to facilitate understanding.

#### tweets_json table
7. To match the other tables, the id field should be "tweet id."
8. No need for the retweet_count column.

### Tidiness issues
1. There should be just one column for doggo, floofer, pupper, and puppo in 

2. Create a single dataframe from the three gathered data.

<a id='CD'></a>
## Cleaning Data
In this section, all the data issues noted during the data assessment were fixed.

In [ ]:
# Make copies of original pieces of data

clean_dog_rate = dog_rate.copy()
clean_image_predictions = image_predictions.copy()
clean_tweets_df = tweets_df.copy()

In [ ]:
clean_dog_rate.info()

### Quality 1 - dog_rate table: incorrect datatype in the timestamp column.

#### Define
The timestamp column's datatype should be "Datetime" and not "str."

#### Code

In [ ]:
# used .astype on timestamp column to modify the column's datatype
# then reassign it to the column to make the desired change

clean_dog_rate['timestamp'] = clean_dog_rate['timestamp'].astype('datetime64[ns]')

#### Test

In [ ]:
# displaying some information about the clean_dog_rate dataframe
# to determine whether the required change has been made

clean_dog_rate.info()

### Quality 2 - dog_rate table: No need for the replies and retweet rows.

#### Define
The following columns' 78 replies need to be removed:
- 'in_reply_to_status_id'
- 'in_reply_to_user_id'

The following columns' 181 retweet need to be removed:
- 'retweeted_status_id'
- 'retweeted_status_user_id', and
- 'retweeted_status_timestamp'

#### Code

In [ ]:
#Remove replies and retweeted tweets 

clean_dog_rate = clean_dog_rate[clean_dog_rate.in_reply_to_status_id.isnull()]
clean_dog_rate = clean_dog_rate[clean_dog_rate.in_reply_to_user_id.isnull()]
clean_dog_rate = clean_dog_rate[clean_dog_rate.retweeted_status_id.isnull()]
clean_dog_rate = clean_dog_rate[clean_dog_rate.retweeted_status_user_id.isnull()]
clean_dog_rate = clean_dog_rate[clean_dog_rate.retweeted_status_timestamp.isnull()]

#### Test

In [ ]:
# displaying some information about the clean_dog_rate dataframe
# to determine whether the required change has been made

clean_dog_rate.info()

### Quality 3 - dog_rate table: No need for the replies and retweet columns.

#### Define
The following columns must be removed because they are not necessary for this analysis:
- 'in_reply_to_status_id'
- 'in_reply_to_user_id'
- 'retweeted_status_id'
- 'retweeted_status_user_id', and
- 'retweeted_status_timestamp'

#### Code

In [ ]:
# used .drop to remove the aforementioned columns
# used inplace = True to effect the change in the main copied data

clean_dog_rate.drop(['in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'], axis=1, inplace=True)

#### Test

In [ ]:
# displaying the list of the columns in clean_dog_rate dataframe
# to determine whether the required change has been made

list(clean_dog_rate.columns)
# clean_dog_rate.info()

### Quality 4 - dog_rate table: In rating_denominator, some values are less than 10.

#### Define
Replacing the values less than 10 in rating_denominator to 10

#### Code

In [ ]:
# used .loc to find values in the rating_denominator column that were less than 10
# then replace them with 10

clean_dog_rate.loc[clean_dog_rate["rating_denominator"] < 10, "rating_denominator"] = 10

#### Test

In [ ]:
# checking the number of less-than-10 entries in the rating_denominator column
# to determine whether the required change has been made

(clean_dog_rate.rating_denominator < 10).sum()

### Quality 5 - image_predictions table: The names in the P columns have some lowercase and some uppercase initials.

#### Define
The first letter will be capitalized for all P columns.

#### Code

In [ ]:
# used .title to change the values in p1, p2 and p3 to title case
# then reassign it to the column to make the desired change

clean_image_predictions.p1 = clean_image_predictions.p1.str.title()
clean_image_predictions.p2 = clean_image_predictions.p2.str.title()
clean_image_predictions.p3 = clean_image_predictions.p3.str.title()

#### Test

In [ ]:
# displaying few top rows from the image_predictions dataframe
# to determine whether the required change has been made

clean_image_predictions.head()

### Quality 6 - The conf should be written completely to facilitate understanding.

#### Define
Using the rename method to change the conf of the P column to confident.

#### Code

In [ ]:
# Creating a dictionary for the old_name as the key and new_name as the value
dict = {'p1_conf': 'p1_confident',
        'p2_conf': 'p2_confident',
        'p3_conf': 'p3_confident'}
 
# call rename () method and pass in the dictionary ceated above
clean_image_predictions.rename(columns=dict, inplace=True)

#### Test

In [ ]:
# displaying column names for image_predictions dataframe
# to determine whether the required change has been made

clean_image_predictions.columns

### Quality 7 - tweets_json table: To match the other tables, the id field should be "tweet id."

#### Define
modify the name of the id column in tweets_json table to 'tweet_id'

#### Code

In [ ]:
# used .rename method to change cloumn name from 'id' to 'tweet_id'

clean_tweets_df.rename(columns = {'id' : "tweet_id"}, inplace= True)

#### Test

In [ ]:
# displaying some information about the clean_tweets_df dataframe
# to determine whether the required change has been made

clean_tweets_df.info()

### Quality 8 - tweets_json table: No need for the retweet_count column.

#### Define
Drop the retweet_count column

#### Code

In [ ]:
clean_tweets_df.drop('retweet_count', axis=1, inplace=True)

#### Test

In [ ]:
# displaying column names for clean_tweets_df dataframe
# to determine whether the required change has been made

clean_tweets_df.columns

### Tidiness 1 -  dog_rate table: There should be just one column for doggo, floofer, pupper, and puppo in dog_rate table

#### Define
Create a single column to represent the dog stage, then eliminate all the other columns that symbolizes the various dog stages.

#### Code

In [ ]:
# Extract the various dog stages from the text column
# utilize it to create a dog stage after that.
clean_dog_rate["dog_stage"] = clean_dog_rate['text'].str.extract('(doggo|pupper|floofer|puppo)')

#drop all four columns
clean_dog_rate.drop(columns=['doggo', 'pupper', 'floofer', 'puppo'], inplace = True)

#### Test

In [ ]:
# displaying the top row from the clean_dog_rate dataframe
# to determine whether the required change has been made

clean_dog_rate.head(1)

In [ ]:
# Display the value count for each dog_stage
clean_dog_rate.dog_stage.value_counts()

### Tidiness 2: Create a single dataframe from the three gathered data.

#### Define
Combine the first and second dataframes using the reduce function, and then use the result to combine it with the third dataframe.

#### Code

In [ ]:
# define list of dataframes
dfs = [clean_dog_rate, clean_image_predictions, clean_tweets_df]

# merge all dataframes into one
merged_df = reduce(lambda  left,right: pd.merge(left,right,on=['tweet_id'], how='left'), dfs)

#### Test

In [ ]:
# displaying few top rows from the merged dataframe

merged_df.head()

In [ ]:
# inspecting the column names for the merged dataframe

merged_df.columns

<a id='SD'></a>
## Storing Data
The cell below saved the gathered, assessed, and cleaned master dataset to a CSV file named "twitter_archive_master.csv".

In [ ]:
# Saving the cleaned and merged dataframe

merged_df.to_csv('twitter_archive_master.csv', index = False)

<a id='AVD'></a>
## Analyzing and Visualizing Data
The analysis and visualization of the wrangled data were done in this section. At least three (3) insights and one (1) visualization should result from this.

In [ ]:
# displaying some information about the merged dataframe

merged_df.info()

In [ ]:
# displaying some statistics about the merged dataframe

merged_df.describe()

### Visualization

In [ ]:
# Displaying value counts for the unique values in dog_stage

merged_df.dog_stage.value_counts()

In [ ]:
# Plotting and labelling a chart for the favorite dog stages

plt.bar(x= merged_df['dog_stage'].value_counts().index, height= merged_df.dog_stage.value_counts().values);
plt.title("Favorite Dog Stages")
plt.xlabel('Dog stages');
plt.ylabel('Frequency');

In [ ]:
merged_df.name.value_counts()[0:11]

In [ ]:
# Plotting and labelling a chart for the top 5 Dog Names in Use
# Excluding weird names like 'a'

merged_df.name.value_counts()[2:7].plot(kind='bar');
plt.title('Top 5 Dog Names in Use');
plt.xlabel('Dog Names');
plt.ylabel('Frequency');
plt.xticks(rotation=0);

### Insights:
1. The most popular Dog stage is Pupper, and the least popular Dog stage is Floofer.
2. The most popular dog names are Charlie and Lucy, which are equally popular.
3. Lucy, Charlie, Oliver, Coper, and Penny are the top 5 most popular dog names respectively.